In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ablation-study-data/ml_enron.txt
/kaggle/input/ablation-study-data/CollegeMsg.txt


In [2]:
!pip install node2vec

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 39.6 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.0
    Uninstalling scipy-1.14.0:
      Successfully uninstalled scipy-1.14.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
libpysal 4.9.2 requires packaging>=22, but you have packaging 21.3 which is incompatible.
libpysal 4.9.2 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
momepy 0.7.2 requires shapely>=2, but you have shapely 1.8.5.post1 which is incompatible.
pointpats 2.5.0 requires shapely>=2, but you have shapely 1.8.5.post1 which is incompatible.
spaghetti 1.7.6 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
spopt 0.6.1 requires shapely>=2.0.1, b

In [3]:
import random
import networkx as nx
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.cluster import KMeans
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score , roc_auc_score, roc_curve
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization,Bidirectional, Dropout, Activation,SimpleRNN ,LSTM , TimeDistributed,Input, LayerNormalization, MultiHeadAttention, GlobalAveragePooling1D, Add
from keras.optimizers import Adam,SGD
from keras.optimizers import AdamW
from keras.layers import LeakyReLU
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras import layers, models, optimizers, activations
from node2vec import Node2Vec
from sklearn.metrics import average_precision_score

In [4]:
class TemporalRandomWalkSampler:
    def __init__(self, filename=None, num_walks=3, alpha=0.5):
        self.num_walks = num_walks
        self.alpha = alpha
        if filename:
            self.temporal_network = self.load_temporal_network(filename)
        else:
            self.temporal_network = nx.Graph()

    def load_temporal_network(self, filename):
        temporal_network = nx.Graph()
        with open(filename, 'r') as file:
            next(file)  # Skip header if there is one
            for line in file:
                parts = line.strip().split(' ')
                if len(parts) == 3:
                    source, target, timestamp = parts
                    temporal_network.add_edge(source, target, time=int(timestamp))
                else:
                    print(f"Skipping invalid line: {line.strip()}")
        return temporal_network

    def sample_temporal_random_walks(self):
        node_sets = {}
        for node in self.temporal_network.nodes():
            walks = self._sample_trws(node)
            node_sets[node] = walks
        return node_sets

    def _sample_trws(self, start_node):
        walks = []
        for _ in range(self.num_walks):
            walk = self.temporal_walk(self.temporal_network, start_node)
            walks.append(walk)
        return walks

    def temporal_walk(self, G, start_node, L=7):
        current_node = start_node
        walk = [current_node]
        current_time = 0

        for _ in range(1, L + 1):
            neighbors = [(neighbor, G.edges[current_node, neighbor]['time']) for neighbor in G.neighbors(current_node)]
            valid_edges = [(neighbor, time) for neighbor, time in neighbors if time > current_time]

            if valid_edges:
                next_node, interaction_time = self.biased_edge_selection(valid_edges, current_time)
                walk.append(interaction_time)
                walk.append(next_node)
                current_node = next_node
                current_time = interaction_time
            else:
                break

        return walk

    def biased_edge_selection(self, valid_edges, current_time):
        weights = [np.exp(-self.alpha * (edge[1] - current_time)) for edge in valid_edges]
        sum_weights = np.sum(weights)

        if sum_weights == 0 or np.isnan(sum_weights):
            # Handle the case when all weights are zero or NaN
            return random.choice(valid_edges)
        else:
            normalized_weights = weights / sum_weights
            selected_index = np.random.choice(len(valid_edges), p=normalized_weights)
            return valid_edges[selected_index]


In [5]:
def extract_temporal_motifs(walks, motif_sizes):
    motifs = {}
    for size in motif_sizes:
        size_motifs = set()
        for walk in walks:
            for i in range(0, len(walk) - size * 2 + 1, 2):
                motif = tuple(walk[i:i + size * 2:2])
                size_motifs.add(motif)
        motifs[size] = size_motifs
    return motifs

def anonymize_motifs(motifs):
    node_encoder = OneHotEncoder()
    flattened_motifs = [str(node) for size in motifs for motif in motifs[size] for node in motif]
    node_encoder.fit(np.array(flattened_motifs).reshape(-1, 1))

    anonymized_motifs = {}
    for size, size_motifs in motifs.items():
        anonymized_size_motifs = set()
        for motif in size_motifs:
            encoded_motif = tuple(tuple(node_encoder.transform(np.array([[str(node)]]).reshape(-1, 1)).toarray().flatten()) for node in motif)
            anonymized_size_motifs.add(encoded_motif)
        anonymized_motifs[size] = anonymized_size_motifs

    return anonymized_motifs

def create_incidence_matrices(motifs):
    incidence_matrices = {}

    for size, size_motifs in motifs.items():
        vertices = set()
        hyperedges = []

        for motif in size_motifs:
            vertices.update(motif)
            hyperedges.append(motif)

        vertices = sorted(vertices)
        num_vertices = len(vertices)
        num_hyperedges = len(hyperedges)

        incidence_matrix = np.zeros((num_vertices, num_hyperedges))

        for j, hyperedge in enumerate(hyperedges):
            for vertex in hyperedge:
                i = vertices.index(vertex)
                incidence_matrix[i, j] = 1

        incidence_matrices[size] = (incidence_matrix, vertices, hyperedges)

    return incidence_matrices

def extract_node_features(temporal_network, incidence_matrices):
    node_features = {}

    for size, (incidence_matrix, vertices, hyperedges) in incidence_matrices.items():
        # Extract features from the incidence matrix for each node
        node_degree = incidence_matrix.sum(axis=1)  # Degree centrality
        node_closeness = np.reciprocal(incidence_matrix.sum(axis=1))  # Closeness centrality
        node_betweenness = np.sum(incidence_matrix, axis=1)  # Betweenness centrality

        # Combine the features into a feature matrix
        features = np.column_stack((node_degree, node_closeness, node_betweenness))

        # Store the features in a dictionary with node labels as keys
        node_features[size] = {node: feature for node, feature in zip(vertices, features)}

    # Extract Node2Vec embeddings
    node2vec = Node2Vec(temporal_network, dimensions=64, walk_length=30, num_walks=200, workers=4)
    model = node2vec.fit(window=10, min_count=1, batch_words=4)
    node2vec_features = {str(node): model.wv[str(node)] for node in temporal_network.nodes()}

    # Merge all features
    merged_features = {}
    for node in temporal_network.nodes():
        embedding_features = node2vec_features[str(node)]
        if node in node_features:
            basic_features = np.concatenate([features[node] for size, features in node_features.items() if node in features])
            merged_features[node] = np.concatenate((basic_features, embedding_features))
        else:
            merged_features[node] = embedding_features

    return merged_features

In [6]:
def prepare_link_prediction_data(temporal_network, node_features, train_size=0.7, val_size=0.15):
    positive_samples = []
    negative_samples = []

    nodes = list(temporal_network.nodes())

    # Create positive samples
    for edge in temporal_network.edges():
        source, target = edge
        if source in node_features and target in node_features:
            positive_samples.append((source, target, node_features[source], node_features[target], 1))

    # Create negative samples
    while len(negative_samples) < len(positive_samples):
        source, target = random.sample(nodes, 2)
        if not temporal_network.has_edge(source, target):
            if source in node_features and target in node_features:
                negative_samples.append((source, target, node_features[source], node_features[target], 0))

    samples = positive_samples + negative_samples
    random.shuffle(samples)

    # Check for inconsistent feature lengths
    max_len = max(len(s[2]) for s in samples)
    print(f"Maximum feature length: {max_len}")
    for s in samples:
        print(f"Feature lengths: {len(s[2])}, {len(s[3])}")

    # Pad or trim features to ensure uniform length
    X = np.array([np.concatenate((pad_or_trim(s[2], max_len), pad_or_trim(s[3], max_len))) for s in samples])
    y = np.array([s[4] for s in samples])
    edge_info = [(s[0], s[1]) for s in samples]

    # Normalize the features
    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    # Split data into training, validation, and testing sets using StratifiedShuffleSplit
    stratified_split = StratifiedShuffleSplit(n_splits=1, test_size=(1 - train_size), random_state=42)

    for train_index, temp_index in stratified_split.split(X, y):
        X_train, X_temp = X[train_index], X[temp_index]
        y_train, y_temp = y[train_index], y[temp_index]
        edge_info_train, edge_info_temp = [edge_info[i] for i in train_index], [edge_info[i] for i in temp_index]

    print("Train, Validation, and Test splits created.")

    # Further split the temp data into validation and test sets
    stratified_split = StratifiedShuffleSplit(n_splits=1, test_size=(val_size / (1 - train_size)), random_state=42)

    for val_index, test_index in stratified_split.split(X_temp, y_temp):
        X_val, X_test = X_temp[val_index], X_temp[test_index]
        y_val, y_test = y_temp[val_index], y_temp[test_index]
        edge_info_val, edge_info_test = [edge_info_temp[i] for i in val_index], [edge_info_temp[i] for i in test_index]

    print("Data preparation complete.")
    return X_train, X_val, X_test, y_train, y_val, y_test, edge_info_train, edge_info_val, edge_info_test


In [7]:
def pad_or_trim(array, length):
    if len(array) < length:
        return np.pad(array, (0, length - len(array)), 'constant')
    else:
        return array[:length]

In [8]:
filename = '/kaggle/input/ablation-study-data/CollegeMsg.txt'  # Replace with your actual file path
filename2 = '/kaggle/input/ablation-study-data/ml_enron.txt'  # Replace with your actual file path

In [9]:
def create_lstm_link_prediction_model(input_shape, activation_function, optimizer):
    model = Sequential([
        Input(shape=input_shape),
        LSTM(128, activation=activation_function, kernel_regularizer=tf.keras.regularizers.l2(0.001)),
        Dropout(0.5),  # Adding dropout to prevent overfitting
        Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
        Dense(1, activation='sigmoid')  # Sigmoid for binary classification (link or no link)
    ])
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model


# **Without Node2vec**

In [13]:
def extract_basic_node_features(temporal_network, incidence_matrices):
    node_features = {}

    for size, (incidence_matrix, vertices, hyperedges) in incidence_matrices.items():
        # Extract features from the incidence matrix for each node
        node_degree = incidence_matrix.sum(axis=1)  # Degree centrality
        node_closeness = np.reciprocal(incidence_matrix.sum(axis=1))  # Closeness centrality
        node_betweenness = np.sum(incidence_matrix, axis=1)  # Betweenness centrality

        # Combine the features into a feature matrix
        features = np.column_stack((node_degree, node_closeness, node_betweenness))

        # Store the features in a dictionary with node labels as keys
        node_features[size] = {node: feature for node, feature in zip(vertices, features)}

    # Merge all features across different motif sizes
    merged_features = {}
    for node in temporal_network.nodes():
        if node in node_features:
            basic_features = np.concatenate([features[node] for size, features in node_features.items() if node in features])
            merged_features[node] = basic_features
        else:
            # Handle nodes without features
            merged_features[node] = np.zeros((len(node_features), 3)).flatten()

    return merged_features

In [30]:
temporal_rw_sampler = TemporalRandomWalkSampler(filename2, num_walks=30, alpha=0.5)
# Sample temporal random walks for each node in the graph
node_sets = temporal_rw_sampler.sample_temporal_random_walks()

# Extract temporal motifs from the sampled walks
all_walks = [walk for walks in node_sets.values() for walk in walks]
motif_sizes = [3, 4, 5, 6]
extracted_motifs = extract_temporal_motifs(all_walks, motif_sizes)

# Create incidence matrices for each n-node motif type
incidence_matrices = create_incidence_matrices(extracted_motifs)

# Extract node features from the incidence matrices (without Node2Vec embeddings)
basic_node_features = extract_basic_node_features(temporal_rw_sampler.temporal_network, incidence_matrices)

In [31]:
# Prepare data for link prediction
X_train, X_val, X_test, y_train, y_val, y_test, edge_info_train, edge_info_val, edge_info_test = prepare_link_prediction_data(
    temporal_rw_sampler.temporal_network, basic_node_features
)

Maximum feature length: 12
Feature lengths: 12, 12
Feature lengths: 12, 12
Feature lengths: 12, 12
Feature lengths: 12, 12
Feature lengths: 12, 12
Feature lengths: 12, 12
Feature lengths: 12, 12
Feature lengths: 12, 12
Feature lengths: 12, 12
Feature lengths: 12, 12
Feature lengths: 12, 12
Feature lengths: 12, 12
Feature lengths: 12, 12
Feature lengths: 12, 12
Feature lengths: 12, 12
Feature lengths: 12, 12
Feature lengths: 12, 12
Feature lengths: 12, 12
Feature lengths: 12, 12
Feature lengths: 12, 12
Feature lengths: 12, 12
Feature lengths: 12, 12
Feature lengths: 12, 12
Feature lengths: 12, 12
Feature lengths: 12, 12
Feature lengths: 12, 12
Feature lengths: 12, 12
Feature lengths: 12, 12
Feature lengths: 12, 12
Feature lengths: 12, 12
Feature lengths: 12, 12
Feature lengths: 12, 12
Feature lengths: 12, 12
Feature lengths: 12, 12
Feature lengths: 12, 12
Feature lengths: 12, 12
Feature lengths: 12, 12
Feature lengths: 12, 12
Feature lengths: 12, 12
Feature lengths: 12, 12
Feature lengt

In [32]:
X_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [33]:
# Reshape the input data to 3D (samples, timesteps, input_dim)
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_val = X_val.reshape((X_val.shape[0], 1, X_val.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

In [34]:
# Create LSTM model
model = create_lstm_link_prediction_model(input_shape=X_train.shape[1:], activation_function='tanh', optimizer=SGD(learning_rate=0.01))


In [35]:
# Print model summary
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                   │ (None, 128)            │        78,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 86,657 (338.50 KB)

 Trainable params: 86,657 (338.50 KB)

 Non-trainable params: 0 (0.00 B)

In [36]:
from tensorflow.keras.callbacks import EarlyStopping

# Define early stopping callback to stop training when validation loss stops improving
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), batch_size=32, callbacks=[early_stopping])

Epoch 1/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.5054 - loss: 0.8231 - val_accuracy: 0.4992 - val_loss: 0.8227
Epoch 2/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4953 - loss: 0.8226 - val_accuracy: 0.4992 - val_loss: 0.8222
Epoch 3/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4752 - loss: 0.8221 - val_accuracy: 0.4992 - val_loss: 0.8217
Epoch 4/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4936 - loss: 0.8216 - val_accuracy: 0.4992 - val_loss: 0.8212
Epoch 5/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5050 - loss: 0.8211 - val_accuracy: 0.5008 - val_loss: 0.8207
Epoch 6/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5086 - loss: 0.8206 - val_accuracy: 0.5008 - val_loss: 0.8202
Epoch 7/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4848 - loss: 0.8201 - val_accuracy: 0.5008 - val_loss: 0.8198
Epoch 8/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4843 - loss: 0.8197 - val_accuracy: 0.5008 - v

In [37]:
# Predict probabilities
y_pred_prob = model.predict(X_test).flatten()

# Calculate AUC
auc = roc_auc_score(y_test, y_pred_prob)
print(f'AUC: {auc:.4f}')

# Calculate Average Precision (AP)
ap = average_precision_score(y_test, y_pred_prob)
print(f'AP: {ap:.4f}')

21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
AUC: 0.5000
AP: 0.4992


# **without motif extraction**

In [10]:
def extract_combined_features(temporal_network):
    # Extract Node2Vec embeddings
    node2vec = Node2Vec(temporal_network, dimensions=64, walk_length=30, num_walks=200, workers=4)
    model = node2vec.fit(window=10, min_count=1, batch_words=4)
    node2vec_features = {str(node): model.wv[str(node)] for node in temporal_network.nodes()}

    # Extract basic features (degree centrality, closeness centrality, betweenness centrality)
    degree_centrality = nx.degree_centrality(temporal_network)
    closeness_centrality = nx.closeness_centrality(temporal_network)
    betweenness_centrality = nx.betweenness_centrality(temporal_network)

    # Combine Node2Vec embeddings with basic features
    combined_features = {}
    for node in temporal_network.nodes():
        basic_features = np.array([
            degree_centrality.get(node, 0),
            closeness_centrality.get(node, 0),
            betweenness_centrality.get(node, 0)
        ])
        combined_features[node] = np.concatenate((node2vec_features[str(node)], basic_features))

    return combined_features

In [11]:
temporal_rw_sampler = TemporalRandomWalkSampler(filename2, num_walks=30, alpha=0.5)
# Sample temporal random walks for each node in the graph
node_sets = temporal_rw_sampler.sample_temporal_random_walks()

# Assuming temporal_rw_sampler and temporal_network are already defined
combined_features = extract_combined_features(temporal_rw_sampler.temporal_network)

Computing transition probabilities:   0%|          | 0/184 [00:00<?, ?it/s]

Generating walks (CPU: 4): 100%|██████████| 50/50 [00:06<00:00,  7.75it/s]


In [12]:
X_train, X_val, X_test, y_train, y_val, y_test, edge_info_train, edge_info_val, edge_info_test = prepare_link_prediction_data(
    temporal_rw_sampler.temporal_network, combined_features
)

Maximum feature length: 67
Feature lengths: 67, 67
Feature lengths: 67, 67
Feature lengths: 67, 67
Feature lengths: 67, 67
Feature lengths: 67, 67
Feature lengths: 67, 67
Feature lengths: 67, 67
Feature lengths: 67, 67
Feature lengths: 67, 67
Feature lengths: 67, 67
Feature lengths: 67, 67
Feature lengths: 67, 67
Feature lengths: 67, 67
Feature lengths: 67, 67
Feature lengths: 67, 67
Feature lengths: 67, 67
Feature lengths: 67, 67
Feature lengths: 67, 67
Feature lengths: 67, 67
Feature lengths: 67, 67
Feature lengths: 67, 67
Feature lengths: 67, 67
Feature lengths: 67, 67
Feature lengths: 67, 67
Feature lengths: 67, 67
Feature lengths: 67, 67
Feature lengths: 67, 67
Feature lengths: 67, 67
Feature lengths: 67, 67
Feature lengths: 67, 67
Feature lengths: 67, 67
Feature lengths: 67, 67
Feature lengths: 67, 67
Feature lengths: 67, 67
Feature lengths: 67, 67
Feature lengths: 67, 67
Feature lengths: 67, 67
Feature lengths: 67, 67
Feature lengths: 67, 67
Feature lengths: 67, 67
Feature lengt

In [13]:
# Reshape the input data to 3D (samples, timesteps, input_dim)
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_val = X_val.reshape((X_val.shape[0], 1, X_val.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

In [14]:
# Create LSTM model
model2 = create_lstm_link_prediction_model(input_shape=X_train.shape[1:], activation_function='tanh', optimizer=SGD(learning_rate=0.01))
# Print model summary
model2.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 128)            │       134,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 142,977 (558.50 KB)

 Trainable params: 142,977 (558.50 KB)

 Non-trainable params: 0 (0.00 B)

In [20]:
from tensorflow.keras.callbacks import EarlyStopping
# Define early stopping callback to stop training when validation loss stops improving
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [21]:
# Train the model
history2 = model2.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), batch_size=32, callbacks=[early_stopping])

Epoch 1/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.5124 - loss: 0.9946 - val_accuracy: 0.4947 - val_loss: 0.9886
Epoch 2/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5426 - loss: 0.9842 - val_accuracy: 0.5609 - val_loss: 0.9798
Epoch 3/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5785 - loss: 0.9758 - val_accuracy: 0.5880 - val_loss: 0.9715
Epoch 4/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5892 - loss: 0.9679 - val_accuracy: 0.6256 - val_loss: 0.9631
Epoch 5/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6357 - loss: 0.9568 - val_accuracy: 0.6526 - val_loss: 0.9545
Epoch 6/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6725 - loss: 0.9481 - val_accuracy: 0.6752 - val_loss: 0.9454
Epoch 7/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6546 - loss: 0.9435 - val_accuracy: 0.6962 - val_loss: 0.9363
Epoch 8/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6703 - loss: 0.9317 - val_accuracy: 0.7143 - 

In [22]:
# Predict probabilities
y_pred_prob = model2.predict(X_test).flatten()

# Calculate AUC
auc = roc_auc_score(y_test, y_pred_prob)
print(f'AUC: {auc:.4f}')

# Calculate Average Precision (AP)
ap = average_precision_score(y_test, y_pred_prob)
print(f'AP: {ap:.4f}')

21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
AUC: 0.9482
AP: 0.9446


# **without biased edegs selection**

In [71]:
class TemporalRandomWalkSamplerNoBias:
    def __init__(self, filename=None, num_walks=3, alpha=0.5):
        self.num_walks = num_walks
        self.alpha = alpha
        if filename:
            self.temporal_network = self.load_temporal_network(filename)
        else:
            self.temporal_network = nx.Graph()

    def load_temporal_network(self, filename):
        temporal_network = nx.Graph()
        with open(filename, 'r') as file:
            next(file)  # Skip header if there is one
            for line in file:
                parts = line.strip().split(' ')
                if len(parts) == 3:
                    source, target, timestamp = parts
                    temporal_network.add_edge(source, target, time=int(timestamp))
                else:
                    print(f"Skipping invalid line: {line.strip()}")
        return temporal_network

    def sample_temporal_random_walks(self):
        node_sets = {}
        for node in self.temporal_network.nodes():
            walks = self._sample_trws(node)
            node_sets[node] = walks
        return node_sets

    def _sample_trws(self, start_node):
        walks = []
        for _ in range(self.num_walks):
            walk = self.temporal_walk(self.temporal_network, start_node)
            walks.append(walk)
        return walks

    def temporal_walk(self, G, start_node, L=7):
        current_node = start_node
        walk = [current_node]
        current_time = 0

        for _ in range(1, L + 1):
            neighbors = [(neighbor, G.edges[current_node, neighbor]['time']) for neighbor in G.neighbors(current_node)]
            valid_edges = [(neighbor, time) for neighbor, time in neighbors if time > current_time]

            if valid_edges:
                next_node, interaction_time = self.uniform_edge_selection(valid_edges)
                walk.append(interaction_time)
                walk.append(next_node)
                current_node = next_node
                current_time = interaction_time
            else:
                break

        return walk

    def uniform_edge_selection(self, valid_edges):
        # Randomly select an edge without bias
        selected_index = random.choice(range(len(valid_edges)))
        return valid_edges[selected_index]


In [78]:
temporal_rw_sampler2 = TemporalRandomWalkSamplerNoBias(filename2, num_walks=30, alpha=0.5)
# Sample temporal random walks for each node in the graph
node_sets = temporal_rw_sampler2.sample_temporal_random_walks()

# Extract temporal motifs from the sampled walks
all_walks = [walk for walks in node_sets.values() for walk in walks]
motif_sizes = [3, 4, 5, 6]
extracted_motifs = extract_temporal_motifs(all_walks, motif_sizes)

# Create incidence matrices for each n-node motif type
incidence_matrices = create_incidence_matrices(extracted_motifs)

# Extract node features from the incidence matrices (without Node2Vec embeddings)
node_features = extract_node_features(temporal_rw_sampler2.temporal_network, incidence_matrices)

Computing transition probabilities:   0%|          | 0/184 [00:00<?, ?it/s]

Generating walks (CPU: 4): 100%|██████████| 50/50 [00:06<00:00,  7.71it/s]


In [79]:
X_train, X_val, X_test, y_train, y_val, y_test, edge_info_train, edge_info_val, edge_info_test = prepare_link_prediction_data(
    temporal_rw_sampler2.temporal_network, node_features
)

Maximum feature length: 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengt

In [80]:
# Reshape the input data to 3D (samples, timesteps, input_dim)
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_val = X_val.reshape((X_val.shape[0], 1, X_val.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

In [81]:
# Create LSTM model
model3 = create_lstm_link_prediction_model(input_shape=X_train.shape[1:], activation_function='tanh', optimizer=SGD(learning_rate=0.01))
# Print model summary
model3.summary()

Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_11 (LSTM)                  │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 139,905 (546.50 KB)

 Trainable params: 139,905 (546.50 KB)

 Non-trainable params: 0 (0.00 B)

In [82]:
# Train the model
history3 = model3.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), batch_size=32)

Epoch 1/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.4893 - loss: 0.9855 - val_accuracy: 0.5203 - val_loss: 0.9781
Epoch 2/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5167 - loss: 0.9801 - val_accuracy: 0.5519 - val_loss: 0.9726
Epoch 3/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5522 - loss: 0.9732 - val_accuracy: 0.5774 - val_loss: 0.9673
Epoch 4/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5327 - loss: 0.9722 - val_accuracy: 0.6045 - val_loss: 0.9620
Epoch 5/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5794 - loss: 0.9651 - val_accuracy: 0.6316 - val_loss: 0.9567
Epoch 6/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6074 - loss: 0.9577 - val_accuracy: 0.6526 - val_loss: 0.9512
Epoch 7/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5976 - loss: 0.9574 - val_accuracy: 0.6692 - val_loss: 0.9457
Epoch 8/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6431 - loss: 0.9473 - val_accuracy: 0.6887 - v

In [83]:
# Predict probabilities
y_pred_prob = model3.predict(X_test).flatten()

# Calculate AUC
auc = roc_auc_score(y_test, y_pred_prob)
print(f'AUC: {auc:.4f}')

# Calculate Average Precision (AP)
ap = average_precision_score(y_test, y_pred_prob)
print(f'AP: {ap:.4f}')

21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
AUC: 0.9435
AP: 0.9306


# **without basic node features**

In [39]:
def extract_node2vec_only(temporal_network):
    # Extract Node2Vec embeddings
    node2vec = Node2Vec(temporal_network, dimensions=64, walk_length=30, num_walks=200, workers=4)
    model = node2vec.fit(window=10, min_count=1, batch_words=4)
    node2vec_features = {str(node): model.wv[str(node)] for node in temporal_network.nodes()}
    
    return node2vec_features

In [65]:
temporal_rw_sampler3 = TemporalRandomWalkSampler(filename, num_walks=30, alpha=0.5)
# Sample temporal random walks for each node in the graph
node_sets = temporal_rw_sampler3.sample_temporal_random_walks()

# Extract temporal motifs from the sampled walks
all_walks = [walk for walks in node_sets.values() for walk in walks]
motif_sizes = [3, 4, 5, 6]
extracted_motifs = extract_temporal_motifs(all_walks, motif_sizes)

# Create incidence matrices for each n-node motif type
incidence_matrices = create_incidence_matrices(extracted_motifs)

# Extract node features from the incidence matrices (without Node2Vec embeddings)
node_features = extract_node2vec_only(temporal_rw_sampler3.temporal_network)

Computing transition probabilities:   0%|          | 0/1899 [00:00<?, ?it/s]

Generating walks (CPU: 4): 100%|██████████| 50/50 [01:31<00:00,  1.84s/it]


In [66]:
X_train, X_val, X_test, y_train, y_val, y_test, edge_info_train, edge_info_val, edge_info_test = prepare_link_prediction_data(
    temporal_rw_sampler3.temporal_network, node_features
)

Maximum feature length: 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengths: 64, 64
Feature lengt

In [67]:
# Reshape the input data to 3D (samples, timesteps, input_dim)
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_val = X_val.reshape((X_val.shape[0], 1, X_val.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

In [68]:
# Create LSTM model
model4 = create_lstm_link_prediction_model(input_shape=X_train.shape[1:], activation_function='tanh', optimizer=SGD(learning_rate=0.01))
# Print model summary
model4.summary()

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_9 (LSTM)                   │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 139,905 (546.50 KB)

 Trainable params: 139,905 (546.50 KB)

 Non-trainable params: 0 (0.00 B)

In [69]:
# Train the model
history4 = model4.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), batch_size=32)

Epoch 1/100
606/606 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.5267 - loss: 0.9805 - val_accuracy: 0.6023 - val_loss: 0.9640
Epoch 2/100
606/606 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.5839 - loss: 0.9607 - val_accuracy: 0.6476 - val_loss: 0.9455
Epoch 3/100
606/606 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.6229 - loss: 0.9431 - val_accuracy: 0.6644 - val_loss: 0.9249
Epoch 4/100
606/606 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.6428 - loss: 0.9222 - val_accuracy: 0.6789 - val_loss: 0.9015
Epoch 5/100
606/606 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.6658 - loss: 0.8991 - val_accuracy: 0.6866 - val_loss: 0.8745
Epoch 6/100
606/606 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.6814 - loss: 0.8731 - val_accuracy: 0.6965 - val_loss: 0.8468
Epoch 7/100
606/606 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.6974 - loss: 0.8479 - val_accuracy: 0.7184 - val_loss: 0.8198
Epoch 8/100
606/606 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.7054 - loss: 0.8278 - val_accu

In [70]:
# Predict probabilities
y_pred_prob = model4.predict(X_test).flatten()

# Calculate AUC
auc = roc_auc_score(y_test, y_pred_prob)
print(f'AUC: {auc:.4f}')

# Calculate Average Precision (AP)
ap = average_precision_score(y_test, y_pred_prob)
print(f'AP: {ap:.4f}')

130/130 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
AUC: 0.9891
AP: 0.9858
